# Test Flex Report Refactoring

Test notebook for the flex requests refactoring PR.
This notebook tests the new FlexClient, DateRange, and TokenManager APIs.

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup: Load IB credentials from environment

In [ ]:
import os
import json
from datetime import date

ib_json_str = os.environ["IB_JSON"]
ib_json = json.loads(ib_json_str)

flex_token = ib_json["accounts"][0]["flex_token"]
query_id = ib_json["accounts"][0]["annual"]

print(f"Token (masked): {flex_token[:4]}...")
print(f"Query ID: {query_id}")

## Test 1: New FlexClient API - Basic fetch (no date range)

In [ ]:
from ngv_reports_ibkr.flex_client import FlexClient

# Create a flex client instance
client = FlexClient()

# Fetch flex report XML data
try:
    xml_data = client.fetch_flex_report(token=flex_token, query_id=query_id)
    print(f"✓ Report fetched successfully")
    print(f"  XML data length: {len(xml_data)} bytes")
except Exception as e:
    print(f"✗ Error fetching report: {e}")

## Test 2: New FlexClient API - With custom date range

In [ ]:
from ngv_reports_ibkr.flex_client import DateRange
from datetime import timedelta
from loguru import logger

# Create a date range for last 30 days
end_date = date.today()
start_date = end_date - timedelta(days=30)

date_range = DateRange(from_date=start_date, to_date=end_date)
logger.info(f"Date range: {date_range.from_date} to {date_range.to_date}")
logger.info(f"Query params: {date_range.to_query_params()}")

# Fetch report with date range
try:
    xml_data_ranged = client.fetch_flex_report(token=flex_token, query_id=query_id, date_range=date_range)
    logger.debug(f"\n✓ Report fetched with custom date range")
    logger.debug(f"  XML data length: {len(xml_data_ranged)} bytes")
except Exception as e:
    logger.error(f"✗ Error: {e}")

## Test 3: DateRange validation

In [ ]:
from ngv_reports_ibkr.flex_client import FlexDateRangeError

# Test 3a: Invalid range (exceeds 365 days)
try:
    invalid_range = DateRange(from_date=date(2024, 1, 1), to_date=date(2025, 2, 1))  # More than 365 days
    print("✗ Should have raised FlexDateRangeError")
except FlexDateRangeError as e:
    print(f"✓ Correctly rejected range > 365 days")

# Test 3b: Valid range (exactly 365 days)
try:
    valid_range = DateRange(from_date=date.today() - timedelta(days=365), to_date=date.today())
    print(f"✓ Accepted 365-day range")
except FlexDateRangeError as e:
    print(f"✗ Error: {e}")

# Test 3c: Invalid range (from after to)
try:
    invalid_range = DateRange(from_date=date(2024, 12, 31), to_date=date(2024, 1, 1))
    print("✗ Should have raised FlexDateRangeError")
except FlexDateRangeError as e:
    print(f"✓ Correctly rejected from_date > to_date")

## Test 4: Token management with TokenManager

In [ ]:
from ngv_reports_ibkr.token_manager import TokenManager, TokenInfo

# Create a token manager
manager = TokenManager()

# Register a token
manager.register_token(flex_token, "U1234567")
token_info = manager.get_token_info("U1234567")

if token_info:
    print(f"Token registered for account: {token_info.account_id}")
    print(f"  Masked token: {token_info.masked_token}")
    print(f"  TTL: {token_info.ttl_hours} hours")
    print(f"  Minutes remaining: {token_info.minutes_remaining:.1f}")
    print(f"  Is expired: {token_info.is_expired}")
    print(f"  Is expiring soon (< 30 min): {token_info.is_expiring_soon}")

## Test 5: Load XML and parse with CustomFlexReport

In [ ]:
from ngv_reports_ibkr.custom_flex_report import CustomFlexReport
import xml.etree.ElementTree as ET

# Parse the XML data we fetched (use xml_data_ranged if available, otherwise xml_data)
try:
    # Use the most recently fetched data
    data_to_parse = xml_data_ranged if "xml_data_ranged" in locals() else xml_data

    # Create a CustomFlexReport and load the XML data
    report = CustomFlexReport()
    report.root = ET.fromstring(data_to_parse)

    account_ids = report.account_ids()
    print(f"✓ Report parsed successfully")
    print(f"  Available accounts: {account_ids}")
    print(f"  Number of accounts: {len(account_ids)}")
except NameError as e:
    print(f"✗ Error: XML data not available. Please run Test 1 or Test 2 first.")
    print(f"  {e}")
except Exception as e:
    print(f"✗ Error parsing report: {e}")

## Test 6: Get trades by account

In [ ]:
# Get trades for each account
try:
    if "account_ids" not in locals():
        print("✗ No account_ids available. Please run Test 5 first.")
    elif account_ids:
        for account_id in account_ids:
            try:
                df = report.trades_by_account_id(account_id)
                print(f"\nAccount: {account_id}")
                print(f"  Trades shape: {df.shape}")
                print(f"  Columns (first 5): {df.columns.tolist()[:5]}")
            except Exception as e:
                print(f"  Error getting trades: {type(e).__name__}")
    else:
        print("✗ No accounts found in report")
except Exception as e:
    print(f"✗ Error: {e}")

## Test 7: New error handling with improved exception hierarchy

In [ ]:
from ngv_reports_ibkr.flex_client import (
    FlexClientError,
    FlexRequestError,
    FlexStatementError,
    FlexTokenError,
    FlexTokenExpiredError,
    FlexRetryableError,
    FlexDateRangeError,
)

print("New exception hierarchy:")
print(f"  FlexClientError (base)")
print(f"    ├─ FlexRequestError (SendRequest phase)")
print(f"    ├─ FlexStatementError (GetStatement phase)")
print(f"    ├─ FlexTokenError")
print(f"    │  └─ FlexTokenExpiredError (6-hour TTL exceeded)")
print(f"    ├─ FlexRetryableError (server busy, rate limited)")
print(f"    └─ FlexDateRangeError (invalid date range)")
print(f"\n✓ New exception hierarchy provides better error classification")